In [1]:
%softreset

import network

sta_if = network.WLAN(network.STA_IF)

sta_if.active(True)
print(sta_if.isconnected())


!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!      softreset      !!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

True


In [1]:
%softreset

import usocket

def open_url(url):
    # open url and return socket
    
    try:
        proto, dummy, host, path = url.split("/", 3)
    except ValueError:
        proto, dummy, host = url.split("/", 2)
        path = ""
    if proto == "http:":
        port = 80
    elif proto == "https:":
        import ussl
        port = 443
    else:
        raise ValueError("Unsupported protocol: " + proto)

    if ":" in host:
        host, port = host.split(":", 1)
        port = int(port)

    ai = usocket.getaddrinfo(host, port, 0, usocket.SOCK_STREAM)
    ai = ai[0]
    
    s = usocket.socket(ai[0], ai[1], ai[2])
    try:
        s.connect(ai[-1])
        if proto == "https:":
            s = ussl.wrap_socket(s, server_hostname=host)
        s.write(b"%s /%s HTTP/1.0\r\n" % ('GET', path))
        s.write(b"Host: %s\r\n\r\n" % host)

        l = s.readline()
        # print("line 1:", l)
        l = l.split(None, 2)
        status = int(l[1])
        reason = ""
        if len(l) > 2:
            reason = l[2].rstrip()
        while True:
            l = s.readline()
            if not l or l == b"\r\n":
                break
            # print("line 2:", l)
            if l.startswith(b"Transfer-Encoding:"):
                if b"chunked" in l:
                    raise ValueError("Unsupported " + l)
            elif l.startswith(b"Location:") and not 200 <= status <= 299:
                raise NotImplementedError("Redirects not supported")
        if status != 200:
            raise Exception("Download failed, status={} {}".format(status, reason))
    except OSError:
        s.close()
        raise

    return s


url = 'http://micropython.org/resources/firmware/esp32-idf3-20200902-v1.13.bin'
url = 'https://github.com/micropython/micropython-lib/blob/master/LICENSE'
url = 'https://raw.githubusercontent.com/micropython/micropython-lib/master/LICENSE'
sock = open_url(url)

try:
    buffer = bytearray(256)

    while True:
        sz = sock.readinto(buffer)
        if not sz:
            # eof
            break
        print(buffer.decode(), end="")
finally:
    print("closing socket")
    sock.close()


!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!      softreset      !!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

micropython-lib consists of multiple modules from different sources and
authors. Each module comes under its own licensing terms. Short name of
a license can be found in a file within a module directory (usually
metadata.txt or setup.py). Complete text of each license used is provided
below. Files not belonging to a particular module a provided under MIT
license, unless explicitly stated otherwise.

=============== MIT License ===============

The MIT License (MIT)

Copyright (c) 2013, 2014 micropython-lib contributors

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
